## imports + setup

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
from typing import Optional 

In [3]:
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, max_tokens_to_sample=2048, api_key=os.getenv("ANTHROPIC_API_KEY"))

## brainstorm story ideas

In [4]:
class StoryIdea(BaseModel):
    """Idea for a story."""
    title: str = Field(description="Title of story")
    summary: str = Field(description="2-3 sentence summary of story")

class StoryIdeaList(BaseModel):
    """List of story ideas."""
    ideas: list[StoryIdea] = Field(description="List of story ideas")

idea_llm = llm.with_structured_output(StoryIdeaList)

genre = "sci-fi"
num_ideas = 10
idea_prompt = f"please brainstorm {num_ideas} idea for a {genre} novel"

In [ ]:
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = idea_llm.invoke(idea_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [9]:
idealist = []
for idea in res.ideas:
    idealist.append({"title": idea.title, "summary": idea.summary})

idealist

[{'title': 'Echoes of the Void',
  'summary': 'In a future where humanity has colonized distant planets, a team of explorers discovers an ancient alien artifact that allows them to communicate with echoes of their past selves. As they unravel the mysteries of their own histories, they must confront the consequences of their choices and the impact on their present.'},
 {'title': 'The Last Memory',
  'summary': 'In a world where memories can be bought and sold, a memory thief stumbles upon a cache of memories that reveal a hidden truth about the origins of humanity. As they delve deeper, they must decide whether to expose the truth or protect the fragile society built on lies.'},
 {'title': 'Quantum Rebellion',
  'summary': 'In a dystopian future governed by a totalitarian regime that uses quantum technology to control reality, a group of rebels discovers a way to manipulate quantum states to alter their fates. They must navigate a treacherous landscape of shifting realities to overthrow

In [10]:
selected_idea = 'Starlight Refugees'
# get the selected idea
selected_idea = [idea for idea in idealist if idea["title"] == selected_idea][0]

selected_idea

{'title': 'Starlight Refugees',
 'summary': 'After Earth becomes uninhabitable, a fleet of generation ships embarks on a journey to find a new home among the stars. As they face challenges from within and outside the fleet, they must grapple with the meaning of home and survival in a vast, empty universe.'}

In [7]:
story_name = "starlight"
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")

In [ ]:
# save the idea to a file
with open(f"json/{story_name}/idea.json", "w") as f:
    json.dump(selected_idea, f, indent=4)

## generate chapter outline

In [6]:
# read in idea from file
with open(f"json/{story_name}/idea.json") as f:
    idea = json.load(f)

In [7]:
title = idea["title"]
summary = idea["summary"]

In [ ]:
# create chapter outline
class Character(BaseModel):
    """Character in a story."""
    name: str = Field(description="Name of character")
    description: str = Field(description="Description/role of character in this chapter")
    # group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")

class Location(BaseModel):
    """Location in a story."""
    name: str = Field(description="Name of location")
    description: str = Field(description="Description/role of location in this chapter")

class Chapter(BaseModel):
    """Chapter in a story."""
    chapter: str = Field(description="Title of chapter, e.g. 'Chapter 1: The Beginning'")
    summary: str = Field(description="1 line summary of chapter")
    description: str = Field(description="longer 2-3 sentence summary of chapter")
    importance: float = Field(description="Importance of chapter to story from 0 to 1 (0 = not important, 1 = very important)")
    conflict: float = Field(description="Conflict level of chapter from 0 to 1 (0 = no conflict, 1 = high conflict)")
    # characters: list[Character] = Field(description="List of characters in chapter.")
    # locations: list[Location] = Field(description="List of locations in chapter.")

class ChapterList(BaseModel):
    """List of chapters."""
    ideas: list[Chapter] = Field(description="List of chapters")

chapter_llm = llm.with_structured_output(ChapterList)

num_chapters = 12
num_chars = 20
num_locs = 8

chapter_prompt=f"Please outline {num_chapters} chapters for a novel titled '{title}' with the summary: '{summary}'. There should be at least {num_chars} unique characters in the story and at least {num_locs} unique locations."

In [9]:
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = chapter_llm.invoke(chapter_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [10]:
# convert to json
chapterlist = []
for chapter in res.ideas:
    # chapterlist.append({"title": chapter.title, "summary": chapter.summary, "description": chapter.description, "importance": chapter.importance, "conflict": chapter.conflict, "characters": [{"name": c.name, "description": c.description} for c in chapter.characters], "locations": [{"name": l.name, "description": l.description} for l in chapter.locations]})
    chapterlist.append({"title": chapter.title, "summary": chapter.summary, "description": chapter.description, "importance": chapter.importance, "conflict": chapter.conflict})

len(chapterlist), chapterlist

(12,
 [{'title': 'Chapter 1: The Last Days of Earth',
   'summary': 'As Earth faces its final days, the last generation ships are prepared for launch.',
   'description': 'The chapter introduces the dire state of Earth, showcasing the environmental collapse and societal breakdown. We meet key characters, including Captain Elara, the determined leader of the fleet, and Dr. Malik, a scientist who has dedicated his life to finding a new home. The urgency of their mission is palpable as they say goodbye to their home planet, setting the stage for the journey ahead.',
   'importance': 1.0,
   'conflict': 0.7},
  {'title': 'Chapter 2: The Fleet Assembles',
   'summary': 'The generation ships gather in orbit, each with its own mission and crew.',
   'description': "This chapter explores the diverse backgrounds of the ships' crews, highlighting the tensions and alliances that form among them. We meet characters like engineer Tessa, who struggles with her past, and diplomat Rian, who tries to m

In [11]:
# save the chapter outline to a file
with open(f"json/{story_name}/chapters.json", "w") as f:
    json.dump(chapterlist, f, indent=4)

## generate scene outline

In [12]:
# read in chapter outline from file
with open(f"json/{story_name}/chapters.json") as f:
    chapters = json.load(f)

In [25]:
# fill in scene details
class SceneCharacter(BaseModel):
    """Character in a scene."""
    name: str = Field(description="Name of character")
    role: str = Field(description="1 line summary of role of character in this scene")
    importance: float = Field(description="Importance of character in scene from 0 to 1 (0 = not important, 1 = very important)")
    emotion: str = Field(description="Emotion of character in scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of character in scene from -1 to 1 (-1 = negative, 0 = neutral, 1 = positive)")

class Scene(BaseModel):
    """Scene in a story."""
    title: str = Field(description="Name of scene")
    summary: str = Field(description="1 line summary of scene")
    location: str = Field(description="Location of scene")
    importance: float = Field(description="Importance of scene in chapter from 0 to 1 (0 = not important, 1 = very important)")
    conflict: float = Field(description="Conflict level of scene from 0 to 1 (0 = no conflict, 1 = high conflict)")
    characters: list[SceneCharacter] = Field(description="List of characters in scene")

class SceneList(BaseModel):
    """List of scenes."""
    scenes: list[Scene] = Field(description="List of scenes in this chapter")

scene_llm = llm.with_structured_output(SceneList)

In [26]:
chapter_scenes = []
for i, chapter in enumerate(chapters):
    print(f"{chapter['title']}")
    invalid_input = True
    while invalid_input:
        try:
            prompt = f"You are a novelist writing a novel titled '{title}' about: '{summary}'. Please list all the key scenes that would be in this chapter: {chapter}."
            res = scene_llm.invoke(prompt)
            chapter_scenes.append(res.scenes)
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
    # break

Chapter 1: The Last Days of Earth
Chapter 2: The Fleet Assembles
Chapter 3: The Void of Space
Chapter 4: Echoes of the Past
Chapter 5: The First Encounter
Chapter 6: Divided Loyalties
Chapter 7: The Lost Colony
Chapter 8: A New Home?
Chapter 9: The Great Divide
Chapter 10: The Battle for Survival
Chapter 11: Reflections in the Stars
Chapter 12: A New Beginning


In [27]:
all_scenes = []
for i, chapter in enumerate(chapter_scenes):
    for j, scene in enumerate(chapter):
        new_scene = {}
        new_scene["title"] = scene.title
        new_scene["number"] = j + 1
        new_scene["summary"] = scene.summary
        new_scene["chapter"] = chapters[i]["title"]
        new_scene["location"] = scene.location
        new_scene["importance"] = scene.importance
        new_scene["conflict"] = scene.conflict
        new_scene["characters"] = [{"name": c.name, "role": c.role, "importance": c.importance, "emotion": c.emotion, "sentiment": c.sentiment} for c in scene.characters]
        all_scenes.append(new_scene)

len(all_scenes), all_scenes

(42,
 [{'title': 'The Crumbling World',
   'number': 1,
   'summary': 'A panoramic view of Earth’s devastation as the last generation ships are being prepared for launch.',
   'chapter': 'Chapter 1: The Last Days of Earth',
   'location': 'Earth, various locations',
   'importance': 1.0,
   'conflict': 0.6,
   'characters': [{'name': 'Narrator',
     'role': 'Describes the environmental collapse and societal breakdown.',
     'importance': 1.0,
     'emotion': 'somber',
     'sentiment': -1.0},
    {'name': 'Captain Elara',
     'role': 'Oversees the final preparations for the fleet.',
     'importance': 1.0,
     'emotion': 'determined',
     'sentiment': 0.0},
    {'name': 'Dr. Malik',
     'role': 'Analyzes data on potential habitable planets.',
     'importance': 1.0,
     'emotion': 'anxious',
     'sentiment': -1.0}]},
  {'title': 'The Farewell',
   'number': 2,
   'summary': 'Key characters say their goodbyes to loved ones and the Earth they are leaving behind.',
   'chapter': '

In [ ]:
# save the scene details to a file
with open(f"json/{story_name}/scenes.json", "w") as f:
    json.dump(all_scenes, f, indent=4)

## get character/location lists

In [8]:
# read in scene details from file
with open(f"json/{story_name}/scenes.json") as f:
    scenes = json.load(f)

In [9]:
# list all unique characters
characters = {}
for scene in scenes:
    for character in scene["characters"]:
        if character["name"] not in characters:
            characters[character["name"]] = character["role"]

len(characters), characters

(22,
 {'Narrator': 'Describes the environmental collapse and societal breakdown.',
  'Captain Elara': 'Oversees the final preparations for the fleet.',
  'Dr. Malik': 'Analyzes data on potential habitable planets.',
  'Various Families': 'Express their sorrow and hope as they part ways.',
  'Engineer Tessa': 'Struggling with her past while preparing her ship for launch.',
  'Diplomat Rian': 'Trying to mediate between different ship crews to maintain peace.',
  'Commander Jax': 'Leader of a rival ship, demanding more resources.',
  'First Officer Jax': "Challenging Elara's decisions and advocating for a different approach",
  'Engineer Mira': 'Trying to fix the malfunction while dealing with crew panic',
  "Malik's Family": 'Symbolizes what he lost',
  'Tessa': 'A young engineer with dreams of exploration',
  "Tessa's Parents": 'Represent her motivation to leave Earth',
  'Crew Members': 'Represent diverse backgrounds and regrets',
  'Alien Commander Zorak': 'Leader of the alien vessel,

In [26]:
# fill in character details
class LegendCharacter(BaseModel):
    """Character in a story."""
    name: str = Field(description="Name of character")
    description: str = Field(description="Description/role of character in this story")
    group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")

class LegendCharacterList(BaseModel):
    """List of characters."""
    characters: list[LegendCharacter] = Field(description="List of characters. Only include characters from the original list of characters.")

legendchar_llm = llm.with_structured_output(LegendCharacterList)

# fill in location details
class LegendLocation(BaseModel):
    """Location in a story."""
    name: str = Field(description="Name of location")
    description: str = Field(description="Description/role of location in this story")

class LegendLocationList(BaseModel):
    """List of locations."""
    locations: list[LegendLocation] = Field(description="List of locations. Only include locations from the original list of locations.")

legendloc_llm = llm.with_structured_output(LegendLocationList)

In [11]:
# get character details
char_prompt = f"list of characters: {characters}"
invalid_input = True
while invalid_input:
    try:
        res = legendchar_llm.invoke(char_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [14]:
character_info = []
for character in res.characters:
    character_info.append({"name": character.name, "description": character.description, "group": character.group})

len(character_info), character_info

(22,
 [{'name': 'Narrator',
   'description': 'Describes the environmental collapse and societal breakdown.',
   'group': 'main characters'},
  {'name': 'Captain Elara',
   'description': 'Oversees the final preparations for the fleet.',
   'group': 'main characters'},
  {'name': 'Dr. Malik',
   'description': 'Analyzes data on potential habitable planets.',
   'group': 'main characters'},
  {'name': 'Various Families',
   'description': 'Express their sorrow and hope as they part ways.',
   'group': 'supporting characters'},
  {'name': 'Engineer Tessa',
   'description': 'Struggling with her past while preparing her ship for launch.',
   'group': 'main characters'},
  {'name': 'Diplomat Rian',
   'description': 'Trying to mediate between different ship crews to maintain peace.',
   'group': 'main characters'},
  {'name': 'Commander Jax',
   'description': 'Leader of a rival ship, demanding more resources.',
   'group': 'antagonists'},
  {'name': 'First Officer Jax',
   'description': 

In [15]:
# save the character details to a file
with open(f"json/{story_name}/characters.json", "w") as f:
    json.dump(character_info, f, indent=4)

In [27]:
# list all unique locations
locations = []
for scene in scenes:
    if scene["location"] not in locations:
        locations.append(scene["location"])

len(locations), locations

(33,
 ['Earth, various locations',
  'Launch site',
  'Orbit around Earth',
  'Fleet assembly area',
  'Docking bays of the ships',
  'Space, departing from Earth',
  'Control Room of the flagship',
  "Earth, Dr. Malik's home",
  "Earth, Tessa's childhood home",
  'Common area of the generation ship',
  "Ship's communication room",
  'Bridge of the flagship',
  'Communication room',
  'Space between the two vessels',
  "Fleet's central command room",
  'Fleet Command Center',
  'Bridge of the lead ship',
  'Surface of the new planet',
  'Dense forest on the planet',
  "Fleet's common area",
  'Observation Deck',
  'Main Hall',
  'Docking Bay',
  'Common Area',
  'Main Hall of the Fleet Ship',
  'Resource Storage Bay',
  'Amidst the chaos of the battle',
  'Command Center',
  'Crew Quarters',
  "Elara's Quarters",
  'Main assembly hall of the flagship',
  'Construction site on the new planet',
  'Central plaza of the new settlement'])

In [28]:
# get location details
loc_prompt = f"list of locations: {locations}. story info: {scenes}"
invalid_input = True
while invalid_input:
    try:
        res = legendloc_llm.invoke(loc_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [29]:
location_info = []
for location in res.locations:
    location_info.append({"name": location.name, "description": location.description})

len(location_info), location_info

(33,
 [{'name': 'Earth, various locations',
   'description': 'A panoramic view of Earth’s devastation as the last generation ships are being prepared for launch.'},
  {'name': 'Launch site',
   'description': 'Key characters say their goodbyes to loved ones and the Earth they are leaving behind.'},
  {'name': 'Orbit around Earth',
   'description': "The generation ships arrive in orbit around Earth, each ship's crew preparing for departure."},
  {'name': 'Fleet assembly area',
   'description': 'Conflicts arise between different ship crews over resources and leadership.'},
  {'name': 'Docking bays of the ships',
   'description': 'The crews make last-minute checks and preparations for launch.'},
  {'name': 'Space, departing from Earth',
   'description': 'The fleet launches, leaving Earth behind and venturing into space.'},
  {'name': 'Control Room of the flagship',
   'description': 'The fleet encounters a critical technical malfunction that threatens their journey.'},
  {'name': "Ea

In [30]:
# save the location details to a file
with open(f"json/{story_name}/locations.json", "w") as f:
    json.dump(location_info, f, indent=4)

## save complete story outline

In [31]:
# compile all the information into a single json file

# read in idea
with open(f"json/{story_name}/idea.json") as f:
    idea = json.load(f)

# read in chapters
with open(f"json/{story_name}/chapters.json") as f:
   chapters = json.load(f)

# read in scenes
with open(f"json/{story_name}/scenes.json") as f:
    scenes = json.load(f)

# read in characters
with open(f"json/{story_name}/characters.json") as f:
    characters = json.load(f)

# read in locations
with open(f"json/{story_name}/locations.json") as f:
    locations = json.load(f)

story_outline = {
    "title": idea["title"],
    "type": "Book",
    "author": "ChatGPT",
    "year": 2024,
    "num_chapters": len(chapters),
    "num_scenes": len(scenes),
    "num_characters": len(characters),
    "num_locations": len(locations),
    "chapters": chapters,
    "scenes": scenes,
    "characters": characters,
    "locations": locations   
}

story_outline

{'title': 'Starlight Refugees',
 'type': 'Book',
 'author': 'ChatGPT',
 'year': 2024,
 'num_chapters': 12,
 'num_scenes': 42,
 'num_characters': 22,
 'num_locations': 33,
 'chapters': [{'title': 'Chapter 1: The Last Days of Earth',
   'summary': 'As Earth faces its final days, the last generation ships are prepared for launch.',
   'description': 'The chapter introduces the dire state of Earth, showcasing the environmental collapse and societal breakdown. We meet key characters, including Captain Elara, the determined leader of the fleet, and Dr. Malik, a scientist who has dedicated his life to finding a new home. The urgency of their mission is palpable as they say goodbye to their home planet, setting the stage for the journey ahead.',
   'importance': 1.0,
   'conflict': 0.7},
  {'title': 'Chapter 2: The Fleet Assembles',
   'summary': 'The generation ships gather in orbit, each with its own mission and crew.',
   'description': "This chapter explores the diverse backgrounds of the 

In [32]:
# save the story outline to a file
with open(f"json/{story_name}/story_outline.json", "w") as f:
    json.dump(story_outline, f, indent=4)

In [33]:
# check if any locations are missing from scenes

# Load the JSON data from a file
with open(f"json/{story_name}/story_outline.json", "r") as file:
    data = json.load(file)

# Extract the locations from the locations object
all_locations = {location["name"] for location in data["locations"]}

# Extract the locations used in scenes
scene_locations = {scene["location"] for scene in data["scenes"]}

# Check if each location in "locations" is included in "scenes"
missing_locations = all_locations - scene_locations

# also other way around
extra_locations = scene_locations - all_locations

# Display the result
if missing_locations:
    print("The following locations are not included in any scene:")
    for location in missing_locations:
        print(f"- {location}")
else:
    print("All locations are included in at least one scene.")

if extra_locations:
    print("The following locations are not defined but used in scenes:")
    for location in extra_locations:
        print(f"- {location}")
else:
    print("All locations used in scenes are defined.")

All locations are included in at least one scene.
All locations used in scenes are defined.


In [34]:
# check characters too
all_characters = {character["name"] for character in data["characters"]}
scene_characters = {character["name"] for scene in data["scenes"] for character in scene["characters"]}
missing_characters = all_characters - scene_characters
extra_characters = scene_characters - all_characters

if missing_characters:
    print("\nThe following characters are not included in any scene:")
    for character in missing_characters:
        print(f"- {character}")

if extra_characters:
    print("\nThe following characters are not defined but used in scenes:")
    for character in extra_characters:
        print(f"- {character}")

In [44]:
# count characters/locations in scenes
with open(f'json/{story_name}/story_outline.json', 'r') as file:
    story_outline = json.load(file)

scenes = story_outline['scenes']
chapters = story_outline['chapters']
characters = story_outline['characters']
locations = story_outline['locations']

In [45]:
# count how many scenes each character is in per chapter
character_scene_count = {}

for scene in scenes:
    chapter = scene['chapter']  # Ensure this is a string or integer
    if chapter not in character_scene_count:
        character_scene_count[chapter] = {}
        
    for character in scene['characters']:
        character_name = character['name']  # Extract a unique identifier from the character dictionary
        if character_name not in character_scene_count[chapter]:
            character_scene_count[chapter][character_name] = 1
        else:
            character_scene_count[chapter][character_name] += 1

character_scene_count

{'Chapter 1: The Last Days of Earth': {'Narrator': 1,
  'Captain Elara': 2,
  'Dr. Malik': 2,
  'Various Families': 1},
 'Chapter 2: The Fleet Assembles': {'Captain Elara': 5,
  'Engineer Tessa': 5,
  'Diplomat Rian': 5,
  'Commander Jax': 1},
 'Chapter 3: The Void of Space': {'Captain Elara': 1,
  'First Officer Jax': 1,
  'Engineer Mira': 1},
 'Chapter 4: Echoes of the Past': {'Dr. Malik': 4,
  "Malik's Family": 1,
  'Tessa': 4,
  "Tessa's Parents": 1,
  'Crew Members': 3},
 'Chapter 5: The First Encounter': {'Captain Elara': 5,
  'Diplomat Rian': 4,
  'Alien Commander Zorak': 1},
 'Chapter 6: Divided Loyalties': {'Captain Elara': 1,
  'First Officer Jax': 1,
  'Tessa': 1},
 'Chapter 7: The Lost Colony': {'Captain Elara': 1, 'First Officer Jax': 1},
 'Chapter 8: A New Home?': {'Captain Reyes': 3,
  'Dr. Malik': 5,
  'Tessa': 5,
  'Crew Member 1': 1,
  'Crew Member 2': 1},
 'Chapter 9: The Great Divide': {'Captain Elara': 4,
  'First Officer Jax': 4,
  'Tessa': 4,
  'Crew Members': 3,

In [46]:
# repeat for locations
location_scene_count = {}

for scene in scenes:
    chapter = scene['chapter']
    if chapter not in location_scene_count:
        location_scene_count[chapter] = {}
        
    location_name = scene['location']
    if location_name not in location_scene_count[chapter]:
        location_scene_count[chapter][location_name] = 1
    else:
        location_scene_count[chapter][location_name] += 1

location_scene_count

{'Chapter 1: The Last Days of Earth': {'Earth, various locations': 1,
  'Launch site': 1},
 'Chapter 2: The Fleet Assembles': {'Orbit around Earth': 1,
  'Fleet assembly area': 2,
  'Docking bays of the ships': 1,
  'Space, departing from Earth': 1},
 'Chapter 3: The Void of Space': {'Control Room of the flagship': 1},
 'Chapter 4: Echoes of the Past': {"Earth, Dr. Malik's home": 1,
  "Earth, Tessa's childhood home": 1,
  'Common area of the generation ship': 2,
  "Ship's communication room": 1},
 'Chapter 5: The First Encounter': {'Bridge of the flagship': 2,
  'Communication room': 2,
  'Space between the two vessels': 1},
 'Chapter 6: Divided Loyalties': {"Fleet's central command room": 1},
 'Chapter 7: The Lost Colony': {'Fleet Command Center': 1},
 'Chapter 8: A New Home?': {'Bridge of the lead ship': 2,
  'Surface of the new planet': 1,
  'Dense forest on the planet': 1,
  "Fleet's common area": 1},
 'Chapter 9: The Great Divide': {'Observation Deck': 1,
  'Main Hall': 1,
  'Dock

In [54]:
# update counts in chapter json object
for chapter in chapters:
    chap_name = chapter['chapter']
    print(chap_name)

    # add character counts
    chap_characters = character_scene_count[chap_name]
    new_char_dict = {}
    for char in chap_characters:
        # find char obj in characters with matching name
        char_obj = [c for c in characters if c["name"] == char][0]
        new_char_dict[char] = {"count": chap_characters[char], "role": char_obj["description"]}
   
    # add location counts
    chap_locations = location_scene_count[chap_name]
    new_loc_dict = {}
    for loc in chap_locations:
        # find loc obj in locations with matching name
        loc_obj = [l for l in locations if l["name"] == loc][0]
        new_loc_dict[loc] = {"count": chap_locations[loc], "role": loc_obj["description"]}

    chapter['characters'] = new_char_dict
    chapter['locations'] = new_loc_dict

chapters

Chapter 1: The Last Days of Earth
Chapter 2: The Fleet Assembles
Chapter 3: The Void of Space
Chapter 4: Echoes of the Past
Chapter 5: The First Encounter
Chapter 6: Divided Loyalties
Chapter 7: The Lost Colony
Chapter 8: A New Home?
Chapter 9: The Great Divide
Chapter 10: The Battle for Survival
Chapter 11: Reflections in the Stars
Chapter 12: A New Beginning


[{'summary': 'As Earth faces its final days, the last generation ships are prepared for launch.',
  'description': 'The chapter introduces the dire state of Earth, showcasing the environmental collapse and societal breakdown. We meet key characters, including Captain Elara, the determined leader of the fleet, and Dr. Malik, a scientist who has dedicated his life to finding a new home. The urgency of their mission is palpable as they say goodbye to their home planet, setting the stage for the journey ahead.',
  'importance': 1.0,
  'conflict': 0.7,
  'chapter': 'Chapter 1: The Last Days of Earth',
  'characters': {'Narrator': {'count': 1,
    'role': 'Describes the environmental collapse and societal breakdown.'},
   'Captain Elara': {'count': 2,
    'role': 'Oversees the final preparations for the fleet.'},
   'Dr. Malik': {'count': 2,
    'role': 'Analyzes data on potential habitable planets.'},
   'Various Families': {'count': 1,
    'role': 'Express their sorrow and hope as they par

In [55]:
# save the updated story outline to a file
with open(f"json/{story_name}/story_outline.json", "w") as f:
    json.dump(story_outline, f, indent=4)

In [65]:
# rank by importance and conflict
with open(f'json/{story_name}/story_outline.json', 'r') as file:
    story_outline = json.load(file)

scenes = story_outline['scenes']
chapters = story_outline['chapters']
characters = story_outline['characters']
locations = story_outline['locations']

In [66]:
# rank each scene by importance
# and within each scene, the characters

for i, chapter in enumerate(chapters):
    importances = []
    conflicts = []
    num_scenes = 0

    chap_name = chapter["chapter"]
    
    # extract importance from each scene
    for j, scene in enumerate(scenes):
        if scene["chapter"] != chap_name:
            continue
        importances.append((j, scene["importance"]))
        conflicts.append((j, scene["conflict"]))
        # now extract character importances
        character_importances = []
        for k, character in enumerate(scene["characters"]):
            character_importances.append((k, character["importance"]))
        # sort character importances
        sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each character
        for k, (l, _) in enumerate(sorted_character_importances):
            scenes[j]["characters"][l]["importance_rank"] = k+1
        # add number of each scene
        scenes[j]["number"] = j+1
        num_scenes += 1
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each scene
    for k, (j, _) in enumerate(sorted_importances):
        scenes[j]["importance_rank"] = k+1
    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each scene
    for k, (j, _) in enumerate(sorted_conflicts):
        scenes[j]["conflict_rank"] = k+1
    
    # add number of scenes to chapter
    chapters[i]["scenes"] = num_scenes
    
    # print results
    # for j, scene in enumerate(scenes):
    #     print(scenes[j]["title"], scenes[j]["importance_rank"])
    #     print("--------------------------------")
    #     for k, character in enumerate(scene["characters"]):
    #         print(character["name"], character["importance_rank"])
    #     print()

In [67]:
scenes

[{'title': 'The Crumbling World',
  'number': 1,
  'summary': 'A panoramic view of Earth’s devastation as the last generation ships are being prepared for launch.',
  'chapter': 'Chapter 1: The Last Days of Earth',
  'location': 'Earth, various locations',
  'importance': 1.0,
  'conflict': 0.6,
  'characters': [{'name': 'Narrator',
    'role': 'Describes the environmental collapse and societal breakdown.',
    'importance': 1.0,
    'emotion': 'somber',
    'sentiment': -1.0,
    'importance_rank': 1},
   {'name': 'Captain Elara',
    'role': 'Oversees the final preparations for the fleet.',
    'importance': 1.0,
    'emotion': 'determined',
    'sentiment': 0.0,
    'importance_rank': 2},
   {'name': 'Dr. Malik',
    'role': 'Analyzes data on potential habitable planets.',
    'importance': 1.0,
    'emotion': 'anxious',
    'sentiment': -1.0,
    'importance_rank': 3}],
  'importance_rank': 1,
  'conflict_rank': 1},
 {'title': 'The Farewell',
  'number': 2,
  'summary': 'Key charac

In [68]:
chapters

[{'summary': 'As Earth faces its final days, the last generation ships are prepared for launch.',
  'description': 'The chapter introduces the dire state of Earth, showcasing the environmental collapse and societal breakdown. We meet key characters, including Captain Elara, the determined leader of the fleet, and Dr. Malik, a scientist who has dedicated his life to finding a new home. The urgency of their mission is palpable as they say goodbye to their home planet, setting the stage for the journey ahead.',
  'importance': 1.0,
  'conflict': 0.7,
  'chapter': 'Chapter 1: The Last Days of Earth',
  'characters': {'Narrator': {'count': 1,
    'role': 'Describes the environmental collapse and societal breakdown.'},
   'Captain Elara': {'count': 2,
    'role': 'Oversees the final preparations for the fleet.'},
   'Dr. Malik': {'count': 2,
    'role': 'Analyzes data on potential habitable planets.'},
   'Various Families': {'count': 1,
    'role': 'Express their sorrow and hope as they par

In [69]:
# save the updated story outline to a file
with open(f"json/{story_name}/story_outline.json", "w") as f:
    json.dump(story_outline, f, indent=4)

## generate text

In [89]:
# read in story outline
with open(f"json/{story_name}/story_outline.json") as f:
    story_outline = json.load(f)

title = story_outline["title"]
chapters = story_outline["chapters"]
scenes = story_outline["scenes"]
chapter_names = [chapter["chapter"] for chapter in chapters]

In [72]:
# create a folder to store the chapter files
if not os.path.exists(f"json/{story_name}/chapters"):
    os.makedirs(f"json/{story_name}/chapters")

In [85]:
for i, chapter in enumerate(chapter_names):
    if i != 6:
        continue
    chap_scenes = [scene for scene in scenes if scene["chapter"] == chapter]
    num_scenes = len(chap_scenes)
    print(f"{chapter}")
    print(f"Number of scenes: {num_scenes}")
    prompt = f"""You are a novelist. Using the story outline below, please write out {chapter}, 
                delineating each scene with "XXX" and making sure each scene has exactly the correct 
                characters (and their respective emotions, importance, etc.), location, and using the 
                chapter and scene summaries to guide your writing. 
                All (and no other) characters and locations should be used.

                Don't explicitly write the scene titles or locations in your output. Each scene should 
                be at least 10 paragraphs long. This chapter should have exactly {num_scenes} scene{'s' if num_scenes > 1 else ''}.

                Here is the story outline:
                {chap_scenes}

                Just output the chapter text (with "XXX" dividers between each scene) and nothing else. 
                Don't add "XXX" at the end of the last scene.
                """
    
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            res = llm.invoke(prompt)
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True

    # save the chapter text to a file
    with open(f"json/{story_name}/chapters/{chapter}.txt", "w") as f:
        f.write(res.content)

    print(f"{chapter} written.")

Chapter 7: The Lost Colony
Number of scenes: 1
Chapter 7: The Lost Colony written.


In [86]:
# check number of scenes in each chapter
for i, chapter in enumerate(chapters):
    # read in chapter text
    with open(f"json/{story_name}/chapters/{chapter['chapter']}.txt") as f:
        chapter_text = f.read()
    
    # count number of scenes (number of 'XXX' dividers + 1)
    num_scenes = chapter_text.count("XXX") + 1

    # compare to expected number of scenes
    if num_scenes == chapter["scenes"]:
        print(f"{chapter['chapter']} has the correct number of scenes.")
    else:
        print(f"{chapter['chapter']} has the wrong number of scenes. Expected {chapter['scenes']}, got {num_scenes}.")

Chapter 1: The Last Days of Earth has the correct number of scenes.
Chapter 2: The Fleet Assembles has the correct number of scenes.
Chapter 3: The Void of Space has the correct number of scenes.
Chapter 4: Echoes of the Past has the correct number of scenes.
Chapter 5: The First Encounter has the correct number of scenes.
Chapter 6: Divided Loyalties has the correct number of scenes.
Chapter 7: The Lost Colony has the correct number of scenes.
Chapter 8: A New Home? has the correct number of scenes.
Chapter 9: The Great Divide has the correct number of scenes.
Chapter 10: The Battle for Survival has the correct number of scenes.
Chapter 11: Reflections in the Stars has the correct number of scenes.
Chapter 12: A New Beginning has the correct number of scenes.


In [88]:
# combine all chapters into a single file
chapter_files = [f"json/{story_name}/chapters/{chapter['chapter']}.txt" for chapter in chapters]

# read in all chapter files
chapter_texts = []
for file in chapter_files:
    with open(file) as f:
        chapter_texts.append(f.read())

chapter_texts

['The sky hung heavy with ash, a muted gray blanket smothering the remnants of what was once a vibrant blue. The air was thick with the scent of decay, a constant reminder of the environmental collapse that had consumed the planet. Cities lay in ruins, their skeletal structures jutting out like broken teeth against the horizon. The Narrator\'s voice echoed in the silence, a somber reflection on the devastation that had become the new normal. \n\nIn the distance, the last generation ships loomed, colossal vessels that promised a flicker of hope in the darkness. Captain Elara stood at the edge of the launch site, her gaze fixed on the ships as they prepared for their final journey. Determination coursed through her veins, a fierce resolve to lead the remnants of humanity into the unknown. She could feel the weight of the world pressing down on her shoulders, but she refused to falter. \n\nDr. Malik was nearby, his brow furrowed in concentration as he analyzed data on potential habitable 

In [90]:
# add chapter title to beginning of each chapter
combined_text = f"{title}\n\n"

for i, chapter in enumerate(chapters):
    combined_text += f"{chapter['chapter']}\n\n"
    combined_text += chapter_texts[i]
    combined_text += "\n\n"

combined_text

'Starlight Refugees\n\nChapter 1: The Last Days of Earth\n\nThe sky hung heavy with ash, a muted gray blanket smothering the remnants of what was once a vibrant blue. The air was thick with the scent of decay, a constant reminder of the environmental collapse that had consumed the planet. Cities lay in ruins, their skeletal structures jutting out like broken teeth against the horizon. The Narrator\'s voice echoed in the silence, a somber reflection on the devastation that had become the new normal. \n\nIn the distance, the last generation ships loomed, colossal vessels that promised a flicker of hope in the darkness. Captain Elara stood at the edge of the launch site, her gaze fixed on the ships as they prepared for their final journey. Determination coursed through her veins, a fierce resolve to lead the remnants of humanity into the unknown. She could feel the weight of the world pressing down on her shoulders, but she refused to falter. \n\nDr. Malik was nearby, his brow furrowed in

In [94]:
# save the combined text to a file
with open(f"scripts/{story_name}.txt", "w") as f:
    f.write(combined_text)

## format story

In [96]:
# read in combined text
with open(f"scripts/{story_name}.txt") as f:
    story_text = f.read()

In [97]:
# replace curly quotes with straight quotes
story_text = story_text.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [102]:
# save the combined text to a file
with open(f"scripts/{story_name}.txt", "w") as f:
    f.write(story_text)

In [103]:
def split_long_lines(filename, folder="scripts", max_length=80):
    # Open the file for reading
    with open(folder + "/" + filename, 'r') as file:
        lines = file.readlines()

    new_lines = []
    
    # Iterate through each line
    for line in lines:
        while len(line) > max_length:
            # Find the last space before the max_length
            split_index = line[:max_length].rfind(' ')
            if split_index == -1:
                split_index = max_length  # No space found, split at max_length
                
            # Append the portion of the line up to the split point
            new_lines.append(line[:split_index].strip() + '\n')
            
            # Continue processing the rest of the line
            line = line[split_index:].strip()
        
        new_lines.append(line + '\n')  # Add the rest of the line (if any) or short lines directly

    # remove any lines that are 'XXX'
    new_lines = [line for line in new_lines if line.strip() != 'XXX']
    
    # Write the modified lines back to the file or a new file
    with open(folder + '/split_' + filename, 'w') as file:
        file.writelines(new_lines)

In [104]:
split_long_lines(f'{story_name}.txt')

In [100]:
def remove_extra_newlines(filename, folder):
    # Open the file for reading
    with open(folder + "/" + filename, 'r') as file:
        lines = file.read()
    
    # Replace 3+ newlines with just 2 newlines
    cleaned_lines = re.sub(r'\n{3,}', '\n\n', lines)

    # Write the cleaned content back to the file (or a new file)
    with open(folder + "/" + filename, 'w') as file:
        file.write(cleaned_lines)

In [105]:
remove_extra_newlines(f'split_{story_name}.txt', 'scripts')

In [106]:
# rename files
# story_name.txt --> story_name_og.txt
# split_story_name.txt --> story_name.txt

os.rename(f"scripts/{story_name}.txt", f"scripts/{story_name}_og.txt")
os.rename(f"scripts/split_{story_name}.txt", f"scripts/{story_name}.txt")

## get line numbers for each scene (after parsing-data.ipynb is run)

In [107]:
def get_scene_boundaries(text_file_path, line_file_path):
    # Read the main text file and find positions of "XXX"
    with open(text_file_path, 'r') as text_file:
        text_lines = text_file.readlines()
    
    # read file with line numbers
    with open(line_file_path, 'r') as line_file:
        line_numbers = line_file.readlines()
    
    # Gather start and end line numbers for each scene based on "XXX" markers
    scene_boundaries = []
    scene_start = 1  # Assuming line numbers start from 1
    for idx, line in enumerate(text_lines):
        if "XXX" in line:
            # find first line after "XXX" that is not empty
            next_line = text_lines[idx + 2][:50]

            # find line number in line_numbers file
            end_index = -1
            for i, line_num in enumerate(line_numbers):
                if next_line in line_num:
                    end_index = i + 1
                    scene_boundaries.append((scene_start, end_index - 1))
                    scene_start = end_index  # Set next scene start after "XXX"
                    idx += 3
                    break
    
    # add last scene boundary
    scene_boundaries.append((scene_start, len(line_numbers)))
    
    
    # Translate scene boundaries to line numbers
    # scene_line_numbers = [(line_numbers[start - 1], line_numbers[end - 1]) for start, end in scene_boundaries]
    
    return scene_boundaries

In [112]:
# Paths to your files
chapter_folder = f'chapters/{story_name}'              # Folder with chapter files
scripts_folder = f'scripts'                         # Folder with script files
text_file_path = f'{scripts_folder}/{story_name}_og.txt'  # File with main text and "XXX"

# read in story_outline.json
with open(f'json/{story_name}/story_outline.json', 'r') as file:
    story_outline = json.load(file)

chapters = story_outline['chapters']
scenes = story_outline['scenes']

chapter_lines = {}

for chapter in chapters:
    chapter_name = chapter['chapter']
    line_file_path = f'{chapter_folder}/{chapter_name}.txt'      # File with line numbers

    # Call the function and display results
    scene_line_numbers = get_scene_boundaries(text_file_path, line_file_path) 

    # Print scene line numbers
    print(chapter_name)
    for idx, (start, end) in enumerate(scene_line_numbers, 1):
        print(f"scene {idx}: starts at line {start}, ends at line {end}")
    print()

    chapter_lines[chapter_name] = scene_line_numbers

Chapter 1: The Last Days of Earth
scene 1: starts at line 1, ends at line 51
scene 2: starts at line 52, ends at line 100

Chapter 2: The Fleet Assembles
scene 1: starts at line 1, ends at line 36
scene 2: starts at line 37, ends at line 65
scene 3: starts at line 66, ends at line 91
scene 4: starts at line 92, ends at line 118
scene 5: starts at line 119, ends at line 143

Chapter 3: The Void of Space
scene 1: starts at line 1, ends at line 55

Chapter 4: Echoes of the Past
scene 1: starts at line 1, ends at line 50
scene 2: starts at line 51, ends at line 98
scene 3: starts at line 99, ends at line 147
scene 4: starts at line 148, ends at line 98
scene 5: starts at line 99, ends at line 233

Chapter 5: The First Encounter
scene 1: starts at line 1, ends at line 40
scene 2: starts at line 41, ends at line 70
scene 3: starts at line 71, ends at line 100
scene 4: starts at line 101, ends at line 40
scene 5: starts at line 41, ends at line 156

Chapter 6: Divided Loyalties
scene 1: start

In [113]:
chapter_lines

{'Chapter 1: The Last Days of Earth': [(1, 51), (52, 100)],
 'Chapter 2: The Fleet Assembles': [(1, 36),
  (37, 65),
  (66, 91),
  (92, 118),
  (119, 143)],
 'Chapter 3: The Void of Space': [(1, 55)],
 'Chapter 4: Echoes of the Past': [(1, 50),
  (51, 98),
  (99, 147),
  (148, 98),
  (99, 233)],
 'Chapter 5: The First Encounter': [(1, 40),
  (41, 70),
  (71, 100),
  (101, 40),
  (41, 156)],
 'Chapter 6: Divided Loyalties': [(1, 65)],
 'Chapter 7: The Lost Colony': [(1, 70)],
 'Chapter 8: A New Home?': [(1, 34),
  (35, 61),
  (62, 87),
  (88, 112),
  (113, 141)],
 'Chapter 9: The Great Divide': [(1, 54), (55, 98), (99, 134), (135, 176)],
 'Chapter 10: The Battle for Survival': [(1, 42),
  (43, 75),
  (76, 102),
  (103, 135)],
 'Chapter 11: Reflections in the Stars': [(1, 33),
  (34, 72),
  (73, 104),
  (105, 137),
  (138, 174)],
 'Chapter 12: A New Beginning': [(1, 36), (37, 66), (67, 97), (98, 127)]}

In [114]:
# add "first_line" and "last_line" to each scene
for chapter in chapters:
    chapter_name = chapter['chapter']
    print(chapter_name)
    scene_line_numbers = chapter_lines[chapter_name]
    print(scene_line_numbers)
    scene_num = 0
    for i, scene in enumerate(scenes):
        compare_chapter = chapter_name
        if scene['chapter'] == compare_chapter:
            print(scene['chapter'], compare_chapter)
            print(scene_num)
            scene['first_line'] = scene_line_numbers[scene_num][0] 
            scene['last_line'] = scene_line_numbers[scene_num][1]
            scene_num += 1

Chapter 1: The Last Days of Earth
[(1, 51), (52, 100)]
Chapter 1: The Last Days of Earth Chapter 1: The Last Days of Earth
0
Chapter 1: The Last Days of Earth Chapter 1: The Last Days of Earth
1
Chapter 2: The Fleet Assembles
[(1, 36), (37, 65), (66, 91), (92, 118), (119, 143)]
Chapter 2: The Fleet Assembles Chapter 2: The Fleet Assembles
0
Chapter 2: The Fleet Assembles Chapter 2: The Fleet Assembles
1
Chapter 2: The Fleet Assembles Chapter 2: The Fleet Assembles
2
Chapter 2: The Fleet Assembles Chapter 2: The Fleet Assembles
3
Chapter 2: The Fleet Assembles Chapter 2: The Fleet Assembles
4
Chapter 3: The Void of Space
[(1, 55)]
Chapter 3: The Void of Space Chapter 3: The Void of Space
0
Chapter 4: Echoes of the Past
[(1, 50), (51, 98), (99, 147), (148, 98), (99, 233)]
Chapter 4: Echoes of the Past Chapter 4: Echoes of the Past
0
Chapter 4: Echoes of the Past Chapter 4: Echoes of the Past
1
Chapter 4: Echoes of the Past Chapter 4: Echoes of the Past
2
Chapter 4: Echoes of the Past Cha

In [115]:
scenes

[{'title': 'The Crumbling World',
  'number': 1,
  'summary': 'A panoramic view of Earth’s devastation as the last generation ships are being prepared for launch.',
  'chapter': 'Chapter 1: The Last Days of Earth',
  'location': 'Earth, various locations',
  'importance': 1.0,
  'conflict': 0.6,
  'characters': [{'name': 'Narrator',
    'role': 'Describes the environmental collapse and societal breakdown.',
    'importance': 1.0,
    'emotion': 'somber',
    'sentiment': -1.0,
    'importance_rank': 1},
   {'name': 'Captain Elara',
    'role': 'Oversees the final preparations for the fleet.',
    'importance': 1.0,
    'emotion': 'determined',
    'sentiment': 0.0,
    'importance_rank': 2},
   {'name': 'Dr. Malik',
    'role': 'Analyzes data on potential habitable planets.',
    'importance': 1.0,
    'emotion': 'anxious',
    'sentiment': -1.0,
    'importance_rank': 3}],
  'importance_rank': 1,
  'conflict_rank': 1,
  'first_line': 1,
  'last_line': 51},
 {'title': 'The Farewell',
 

In [116]:
# update json file
with open(f'json/{story_name}/story_outline.json', 'w') as file:
    json.dump(story_outline, file, indent=4)